# HyLoRADA: Software Evolution & Reengineering

**HyLoRADA**: Hybrid Low-Rank Direct Attention Adaptation

## Novel Contributions
1. **Orthogonal initialization** - Prevents rank collapse
2. **Gated magnitude** - Learnable magnitude control
3. **Residual LoRA path** - Blends DoRA + LoRA dynamics

## Evolution Path
```
LoRA (2021) → DoRA (2024) → HyLoRADA (2026)
```

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'

In [ ]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

In [ ]:
!pip install -q transformers datasets accelerate tqdm optuna

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

---
## 1. Run Benchmark on WikiText

In [ ]:
# Benchmark on WikiText-2 (Language Modeling)
!python run_benchmark.py \
    --model Qwen/Qwen2.5-0.5B \
    --dataset wikitext \
    --max_length 1024 \
    --epochs 3 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_train 1000 \
    --num_test 100 \
    --methods lora dora lorada hylorada \
    --output_dir ./results_wikitext

---
## 2. Run Benchmark on Code (Software Engineering Task)

In [ ]:
# Benchmark on CodeSearchNet (Python Code Summarization)
!python run_benchmark.py \
    --model Qwen/Qwen2.5-0.5B \
    --dataset code \
    --max_length 512 \
    --epochs 3 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_train 1000 \
    --num_test 100 \
    --methods lora dora hylorada \
    --output_dir ./results_code

---
## 3. Generate Qualitative Examples

In [ ]:
# Generate comparison examples (code summarization + lost-in-middle)
!python generate_examples.py \
    --checkpoint_dir ./results_wikitext \
    --output comparison_examples.md

In [ ]:
# View the generated examples
with open('comparison_examples.md', 'r') as f:
    print(f.read())

---
## 4. Hyperparameter Optimization (Optional)

In [ ]:
# Bayesian optimization with Optuna (10 trials)
!python optimize_hylorada.py --n_trials 10 --epochs 2

---
## 5. View Results

In [ ]:
import json
import os

# Load WikiText results
for results_dir in ['./results_wikitext', './results_code']:
    results_file = os.path.join(results_dir, 'results.json')
    if os.path.exists(results_file):
        with open(results_file) as f:
            results = json.load(f)
        print(f"\n=== {results_dir} ===")
        for method, data in results.items():
            if 'perplexity' in data:
                print(f"{method}: PPL={data['perplexity']:.2f}, Params={data.get('trainable_params', 'N/A'):,}")

In [ ]:
# Download results for report
!zip -r hylorada_results.zip results_wikitext results_code comparison_examples.md METHODOLOGY.md